In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

In [2]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(2, activation='softmax')(x)  # 2 classes: healthy and tumor

model = Model(inputs=base_model.input, outputs=x)

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,       
    rotation_range=20,    
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))


test_dir = os.path.join(BASE_DIR, "data", "preprocessed_split_2","test")
train_dir = os.path.join(BASE_DIR, "data", "preprocessed_split_2","train")
val_dir = os.path.join(BASE_DIR, "data", "preprocessed_split_2","val")

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),   # VGG19 input size
    batch_size=32,
    class_mode='categorical'  # because categorical_crossentropy
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 2315 images belonging to 2 classes.
Found 661 images belonging to 2 classes.
Found 333 images belonging to 2 classes.


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)

Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 1002s 14s/step - accuracy: 0.8046 - loss: 0.8519 - val_accuracy: 0.8344 - val_loss: 0.3231
Epoch 2/10
 1/72 ━━━━━━━━━━━━━━━━━━━━ 17:21 15s/step - accuracy: 0.7812 - loss: 0.5215

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


72/72 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.7812 - loss: 0.5215 - val_accuracy: 0.8359 - val_loss: 0.3216
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8876 - loss: 0.2591

In [ ]:
y_true = test_generator.classes
y_pred_probs = model.predict(test_generator, verbose=1)

y_pred = (y_pred_probs > 0.5).astype(int).flatten()

test_acc = accuracy_score(y_true, y_pred)
test_precision = precision_score(y_true, y_pred)
test_recall = recall_score(y_true, y_pred)
test_f1 = f1_score(y_true, y_pred)
test_cm = confusion_matrix(y_true, y_pred)

print("Test Accuracy :", test_acc)
print("Test Precision:", test_precision)
print("Test Recall   :", test_recall)
print("Test F1-Score :", test_f1)
print("Confusion Matrix:\n", test_cm)

In [ ]:
import os

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
model_dir = os.path.join(BASE_DIR, "models")

os.makedirs(model_dir, exist_ok=True)

model.save(os.path.join(model_dir, "VGG19_trained.keras"))